In [ ]:
import pickle
import numpy as np
import msmrd2.analysis as msmrdals
import pyemma
import pyemma.plots as mplt
import matplotlib.pyplot as plt

In [ ]:
# Parameters for loading files
nfiles = 500
dt = 0.00001 # from original simulation
stride = 25  # stride of sampling (every 25th dt)
dtEffective = dt*stride # needed to obtain rate dictionary
totalTimeSteps = 2000000
fnamebase = '../data/dimer/simDimer_t2.00E+06_s25_'
# Parameters for MSM generation 
numBoundStates = 8
lagtime = 200
reversible = True
stitching = True
fullDictionary = False # Only rates to and from bound states.

In [ ]:
# Load discrete trajectories
dtrajs = []
for i in range(nfiles):
    dtraj = msmrdals.loadDiscreteTrajectory(fnamebase, i)
    dtrajs.append(dtraj)    
    print("File ", i+1, " of ", nfiles, " done.", end="\r")

In [ ]:
# Slice trajectories getting rid of the unbound state 0
unboundStateIndex = 0
slicedDtrajs = msmrdals.splitDiscreteTrajs(dtrajs, unboundStateIndex)
# Stitch trajectories if wanted
if stitching:
    finalTrajs = msmrdals.stitchTrajs(slicedDtrajs, 500)
else:
    finalTrajs = slicedDtrajs

In [ ]:
# Create MSM between transision states and bound states without stitching
mainmsm = pyemma.msm.estimate_markov_model(finalTrajs, lagtime, reversible=reversible)
# The active set keep track of the indexes used by pyemma and the ones used to describe the state in our model.
activeSet = mainmsm.active_set
print(activeSet)

In [ ]:
# CK-test
nstates = 8
cktest = mainmsm.cktest(nstates, mlags=6)
mplt.plot_cktest(cktest);

In [ ]:
maxlagtime = 1000
nits = 40
its = pyemma.msm.its(finalTrajs, maxlagtime, reversible=reversible)
mplt.plot_implied_timescales(its, nits = nits, ylog=True, units='steps', linewidth=2, dt=1)
plt.ylabel(r"log(timescale/steps)", fontsize = 24)
plt.xlabel(r"lag time/steps", fontsize = 24)

In [ ]:
mplt.plot_implied_timescales(its, nits = nits, ylog=False, units='steps', linewidth=2, dt=1)
plt.ylabel(r"timescale/steps", fontsize = 24)
plt.xlabel(r"lag time/steps", fontsize = 24)

In [ ]:
eig0 = mainmsm.eigenvectors_right()[4]
np.where(eig0>0.01)

In [ ]:
rateDictionary = msmrdals.MSMtoRateDictionary(mainmsm, numBoundStates, dtEffective, fullDictionary)
# Alternatively, it can all be done with one function: slice and stitch trajectories then build MSM and extract rates
# rateDictionary = msmrdals.extractRatesMSM(dtrajs, lagtime, numBoundStates, stitching, fullDictionary)

In [ ]:
# Export to data file with pickle (to be loaded later)
pickle_out = open("pickled_data/ratedictionary_dimer_test_t" + "{:.2E}".format(totalTimeSteps ) + "_s" + "{:d}".format(stride) + "_lagt" + "{:d}".format(lagtime) + ".pickle","wb")
pickle.dump(rateDictionary, pickle_out)
pickle_out.close()

In [ ]:
rateDictionary

In [ ]:
rateDictionary["b2->b1"]